In [ ]:
using Pkg
# Pkg.activate(".")
# Pkg.instantiate()
# Pkg.add("SQLite")
# Pkg.add("DBInterface")
# Pkg.add("JSON3")
# Pkg.add("EasyConfig")
# Pkg.add("MurmurHash3")
# Pkg.add("TextAnalysis")
# Pkg.add("UUIDs")
# Pkg.add("PooledArrays")
# Pkg.add("CSV")
# Pkg.add("FilePathsBase")
# Pkg.add("HDF5")


include("src/lisa_store.jl")

using SQLite
using DBInterface
using MurmurHash3
using TextAnalysis
using JSON3
using PooledArrays

db = Graph.DB("lisa.db")
hll = SetCore.HllSet{10}()

# Run the following to create the HDF5 file. Do it once and then comment it out
# Create a new empty HDF5 file
# h5open("lisa_arch.hdf5", "w") do file
#     # No data is written to the file
end

In [12]:
p = 10
hll = SetCore.HllSet{p}()
Store.book_file(db, "/home/alexmy/JULIA/DEMO/raw/eu/", hll)

In [13]:
using UUIDs
# parent::String, type::String,processor_id::String, status::String, lock_uuid::String
uuid = string(uuid4())
df = Graph.set_lock!(db, "/home/alexmy/JULIA/DEMO/raw/eu", "csv", "book_file", "ingest_csv", "waiting", "waiting", uuid; result=true)

for row in eachrow(df)
    assign = Graph.Assignment(row.id,row.parent, row.item, row.a_type, row.processor_id, row.status, row.lock_uuid)     
    col_uuid = string(uuid4())
    # db::Graph.DB, file::Graph.Assignment, lock_uuid::String; p::Int=10    
    Store.ingest_csv(db, assign, col_uuid; limit=12000, offset=10)
end

Processed column: 0


Processed column: 0


Processed column: 1


Processed column: 1


Processed column: 2


...

Processed column: 0


Processed column: 0


Processed column: 0
Processed column: 0


Processed column: 2


Processed column: 2


In [14]:
using EasyConfig
Store.commit(db, "lisa_arch.hdf5", "Alex Mylnikov", "alexmy@lisa-park.com", "commit 2", Config())

props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/Italy_traffic_weather/damasco_final.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/601b1268539c8ff94eda6b555e6a1a3cd3a164c4, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/Italy_traffic_weather/stazioni_meteo.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/8b9bd1124d2a5c05d5263fa74616076b1adad006, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/Italy_traffic_weather/TrFl/SmCtCG_TrFl_Str000_lastMonth.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/fd21e67fbcdd68d0e82ea6994f9299a2023f3c78, _csv_file_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/26ec3695fabb1aca90467c4c04c3869daafb6d34, _csv_column_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "Description", "file_sha1" => "a2198d4d216c50945cc7971b0e15349e2e0cdf97", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/c6f372e6a8d1146208e03c527aef0af997c678f7, _csv_column_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/cdc/FRPP_GLC_-_United_States.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/f5c427f989056400cd03aae2e67793564c81117d, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/cdc/NLDAS_daily_precip_by_county_2000_2011.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/62d9485b2a00

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/c792af60a60cf7cf9aa575efd22a396d01addd2e, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/data.gov.uk/uk_companies-p1-2.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/71cc3f0e1ab964a682f29967d085c0aa45b64acd, _csv_file_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "RegAddress.AddressLine1", "file_sha1" => "71cc3f0e1ab964a682f29967d085c0aa45b64acd", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/5b899453043e82bd496ad1d81fb8f719ac79207e, _csv_column_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "SICCode.SicText_1", "file_sha1" => "71cc3f0e1ab964a682f29967d085c0aa45b64acd", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/n

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/431428e61541895457ab3d8d2e78ffbcd26248c0, _csv_column_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "SICCode.SicText_1", "file_sha1" => "e72c078757c6e03c12fc996f668ade892b5120c1", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/34f1d6e161d613918860af5788effbf9c4354d8d, _csv_column_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "URI", "file_sha1" => "e72c078757c6e03c12fc996f668ade892b5120c1", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/71d778b5485103e21f8bc9a4cca0dd2ce1e62895, _csv_column_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/data.gov.uk/uk_companies-p5-2.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/3da13fcace2

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/c3157aee164669eba31fc4d4816fe43a34f34372, _csv_column_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/eu_data/firstnames1.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/0c3f59ddd3d4e9a33773de121a62f456ac3b5388, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/eu_data/netherlands_street.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/d335964d70c276ad46ae37d4be6ced8c7550f939, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/eu_data/netherlands_street_city_date.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/9db34643522cf5341ff34b20bad6324e26326449, _cs

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/2942161ecf436f17202744716925a14562538da4, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/publicdata.eu/uk_empty-homes-borough.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/da812b23b73aa83512396dbd1aa086b03457feb9, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/publicdata.eu/uk_gla-general-elections-votes-party-2010.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/77a3b2acbde9e48d49a0c6142f6cc7cd2cf5df8e, _csv_file_
props: Dict{String, Any}("file_name" => "/home/alexmy/JULIA/DEMO/raw/eu/uk_vehicle_casualties/Isc_london-ksi-only-since2010.csv", "file_type" => "csv", "commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1")String
save_node: /759fba97-e4e6-48e9-86ab-aa023

String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/4b27d9f46b96aac59fa092b7900c7088bfb29daf, _csv_file_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "Vehicle type", "file_sha1" => "4b27d9f46b96aac59fa092b7900c7088bfb29daf", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/5e7427de7d1f2566dd382106f209c48a67ed00f2, _csv_column_
props: Dict{String, Any}("commit_id" => "759fba97-e4e6-48e9-86ab-aa023693a0b1", "column_name" => "Pedestrian Locality", "file_sha1" => "4b27d9f46b96aac59fa092b7900c7088bfb29daf", "column_type" => "String")String
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/nodes/e6f25ad5d838dc49f8bddac019f8f6c7bf1f00c7, _csv_column_
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/fd21e67fbcdd68d0e82ea6994f9299a2023f3c78/has_column, b6bf6c59d0a515e57f0f8fade5a1ba1a5e611b7e
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/ea6d633c8f95e16f2a2fd066f59ffef3af797206/has

save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/71cc3f0e1ab964a682f29967d085c0aa45b64acd/has_column, f7ce156d7df7dbd486cd888c0bd49f4b8bec106d
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/71cc3f0e1ab964a682f29967d085c0aa45b64acd/has_column, 1ba41353130e36a257fb962eb719d58c9c876814
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/e72c078757c6e03c12fc996f668ade892b5120c1/has_column, 431428e61541895457ab3d8d2e78ffbcd26248c0
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/e72c078757c6e03c12fc996f668ade892b5120c1/has_column, 34f1d6e161d613918860af5788effbf9c4354d8d
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/e72c078757c6e03c12fc996f668ade892b5120c1/has_column, 71d778b5485103e21f8bc9a4cca0dd2ce1e62895
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/3da13fcace26bf7993425973090823e88c394be2/has_column, 02b4e972ea698768322c30eff0fab5a64afda302
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/3da13fcace26bf7993425973090823e88c394be2/has_column

save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/1c4c950dda437c3d55ee6ef6a43548484facde02/has_column, 668d871e8b13df7e378375c676168703761fedd3
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/4b27d9f46b96aac59fa092b7900c7088bfb29daf/has_column, 5e7427de7d1f2566dd382106f209c48a67ed00f2
save_node: /759fba97-e4e6-48e9-86ab-aa023693a0b1/edges/4b27d9f46b96aac59fa092b7900c7088bfb29daf/has_column, e6f25ad5d838dc49f8bddac019f8f6c7bf1f00c7


In [15]:
using HDF5
arch = h5open("lisa_arch.hdf5")

🗂️ HDF5.File: (read-only) lisa_arch.hdf5
└─ 📂 759fba97-e4e6-48e9-86ab-aa023693a0b1
   ├─ 📂 edges
   │  ├─ 📂 1c4c950dda437c3d55ee6ef6a43548484facde02
   │  │  └─ 📂 has_column
   │  │     ├─ 🔢 668d871e8b13df7e378375c676168703761fedd3
   │  │     │  └─ (5 children)
   │  │     └─ 🔢 88568fd93f5b196bc5495ff637fcb4ef1dd52a81
   │  │        └─ (5 children)
   │  ├─ 📂 3da13fcace26bf7993425973090823e88c394be2
   │  │  └─ 📂 has_column
   │  │     ├─ 🔢 02b4e972ea698768322c30eff0fab5a64afda302
   │  │     │  └─ (5 children)
   │  │     ├─ 🔢 7bb6a94745236688ae32e5f7013745148f9ffb98
   │  │     │  └─ (5 children)
   │  │     └─ 🔢 c3157aee164669eba31fc4d4816fe43a34f34372
   │  │        └─ (5 children)
   │  ├─ 📂 4b27d9f46b96aac59fa092b7900c7088bfb29daf
   │  │  └─ 📂 has_column
   │  │     ├─ 🔢 5e7427de7d1f2566dd382106f209c48a67ed00f2
   │  │     │  └─ (5 children)
   │  │     └─ 🔢 e6f25ad5d838dc49f8bddac019f8f6c7bf1f00c7
   │  │        └─ (5 children)
   │  ├─ 📂 71cc3f0e1ab964a682f29967d085c0aa45b64a

In [ ]:
close(arch)

In [17]:
using HDF5

file = h5open("lisa_arch.hdf5", "r")
data_out = Dict()
Store.read_datasets(file, data_out, "/759fba97-e4e6-48e9-86ab-aa023693a0b1/")
println(data_out)
# Close the file
close(file)

Dict{Any, Any}(

"f7ce156d7df7dbd486cd888c0bd49f4b8bec106d" => "{\"source\":\"/home/alexmy/JULIA/DEMO/raw/eu/data.gov.uk/uk_companies-p1-2.csv\",\"target\":\"SICCode.SicText_1\",\"source_label\":\"csv_file\",\"target_label\":\"csv_column\",\"commit_id\":\"759fba97-e4e6-48e9-86ab-aa023693a0b1\"}", "5b899453043e82bd496ad1d81fb8f719ac79207e" => "{\"source\":\"/home/alexmy/JULIA/DEMO/raw/eu/data.gov.uk/uk_companies-p1-2.csv\",\"target\":\"RegAddress.AddressLine1\",\"source_label\":\"csv_file\",\"target_label\":\"csv_column\",\"commit_id\":\"759fba97-e4e6-48e9-86ab-aa023693a0b1\"}", "5e7427de7d1f2566dd382106f209c48a67ed00f2" => "{\"source\":\"/home/alexmy/JULIA/DEMO/raw/eu/uk_vehicle_casualties/R1o_london-rtc-since2010-lsoa-msoa-ward.csv\",\"target\":\"Vehicle type\",\"source_label\":\"csv_file\",\"target_label\":\"csv_column\",\"commit_id\":\"759fba97-e4e6-48e9-86ab-aa023693a0b1\"}", "c6f372e6a8d1146208e03c527aef0af997c678f7" => "{\"source\":\"/home/alexmy/JULIA/DEMO/raw/eu/Italy_traffic_weather/TrFl/SmCtS

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4611686018427387904, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2305843009213693952, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

, 0, 0, 4611686018427387904, 6917529027641081856, 0, 0, 4611686018427387904, 5764607523034234880, 0, 6917529027641081856, 0, 0, 0, 4611686018427387904, 4611686018427387904, 4647714815446351872, 0, 0, 4611686018427387904, 4611686018427387904, 2305843009213693952, 4611686018427387904, 0, 3458764513820540928, 1152921504606846976, 1152921504606846976, 0, 2305843009213693952, 0, 0, 4611686018427387904, 288230376151711744, 1152921504606846976, 0, 0, 0, 4611686018427387904, 2305843009213693952, 0, 2305843009213693952, 0, 0, 4611686018427387904, 2305843009213693952, 0, 0, 4611686018427387904, 864691128455135232, 0, 2305843009213693952, 0, 0, 0, 0, 0, 576460752303423488, 0, 432345564227567616, 576460752303423488, 0, 4611686018427387904, 0, 0, 2305843009213693952, 0, 2305843009213693952, 0, 0, 0, 576460752303423488, 1152921504606846976, 2882303761517117440, 1152921504606846976, 2305843009213693952, 0, 6917529027641081856, 0, 2305843009213693952, 4611686018427387904, 2305843009213693952, 11529215

In [ ]:
Graph.unset_lock!(db, :, :)